In [76]:
import pandas as pd
import numpy
import os
import plotly.express as px

df = pd.read_csv('./data/Speed+Dating+Data.csv',low_memory=False,encoding='cp1252')
display(df.head())
display(df.shape)
display(df.columns)
display(df[['age','gender','match']].describe())

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


(8378, 195)

Index(['iid', 'id', 'gender', 'idg', 'condtn', 'wave', 'round', 'position',
       'positin1', 'order',
       ...
       'attr3_3', 'sinc3_3', 'intel3_3', 'fun3_3', 'amb3_3', 'attr5_3',
       'sinc5_3', 'intel5_3', 'fun5_3', 'amb5_3'],
      dtype='object', length=195)

,age,gender,match
count,8283.000000,8378.000000,8378.000000
mean,26.358928,0.500597,0.164717
std,3.566763,0.500029,0.370947
min,18.000000,0.000000,0.000000
25%,24.000000,0.000000,0.000000
50%,26.000000,1.000000,0.000000
75%,28.000000,1.000000,0.000000
max,55.000000,1.000000,1.000000


# Remarques sur les données et data cleaning

1. Il y a une différence de notation entre les groupes sur les batchs 6 à 9 (notation de 1 à 10 au lieu de 100 points à distribuer)
2. Il y a un manque important de données principalement sur le remplissage des forulaires notamment après le speed dating 


Pour que les données restent cohérentes dans l'analyse qui va suivre je vais filtrer les données pour exclure les wave 6 à 9


In [77]:
print(f'Nombre de valeurs vides : \n{df.isna().sum().sort_values(ascending=False).head(10)}')
print(f"{len(df.columns[df.isna().sum() > 1000])} colonnes ont plus de 1000 valeurs manquantes")
print(f"{len(df.columns[df.isna().sum() > 2000])} colonnes ont plus de 2000 valeurs manquantes")
print(f"{len(df.columns[df.isna().sum() > 3000])} colonnes ont plus de 3000 valeurs manquantes")
print(f"{len(df.columns[df.isna().sum() > 4000])} colonnes ont plus de 4000 valeurs manquantes")

list_to_delete = df.columns[df.isna().sum() > 4000].to_list()

filterd_df = df[~df['wave'].isin([6,7,8,9])]
filterd_df = filterd_df.drop(list_to_delete,axis=1)
print(f"shape après le filtrage : {filterd_df.shape}")


Nombre de valeurs vides : 
num_in_3    7710
numdat_3    6882
expnum      6578
sinc7_2     6423
amb7_2      6423
shar7_2     6404
attr7_2     6394
fun7_2      6394
intel7_2    6394
fun5_3      6362
dtype: int64
94 colonnes ont plus de 1000 valeurs manquantes
83 colonnes ont plus de 2000 valeurs manquantes
71 colonnes ont plus de 3000 valeurs manquantes
65 colonnes ont plus de 4000 valeurs manquantes
shape après le filtrage : (6816, 130)



### Variables clés (extrait du Data Key)

- **match** (1/0) : match réciproque (deux "Yes").  
- **dec** / **dec_o** : décision du participant / du partenaire ce soir-là (1=Yes, 0=No).  
- **attr, sinc, intel, fun, amb, shar** : notes attribuées au partenaire lors du *date* (1–10).  
- **attr_o, ...** : notes que **le partenaire** vous attribue.  
- **pf_o_att** : préférences déclarées du partenaire (vagues/échelles différentes selon *wave*).  
- **samerace** : 1 si même origine ethnique, 0 sinon.  
- **age, age_o** : âge du participant / du partenaire.  
- **gender** : 0 = Femme, 1 = Homme.  
- **order** : numéro du rendez-vous dans la soirée.  
- **attr3_1, ..., amb3_1** : **auto-évaluations** (Time 1).  
- **attr1_1, ..., shar1_1** : **importance déclarée** des attributs (Time 1).


On se concentre sur TIME 1 car variabilité trop importante après



In [78]:
print("Distribution de gender")
fig = px.bar(filterd_df['gender'].value_counts().reset_index().replace(1,'Hommes').replace(0,'Femmes'),x='gender',y='count',title='Distribution gender')
fig.show()

Distribution de gender


In [79]:
distrib_metier = px.bar(df['career'].value_counts()[:40].reset_index(),x='career',y='count',title='Distribution career (40 premiers)')
distrib_metier.show()

# Distribution de l'étnie des participants

In [80]:
race = filterd_df['race'].value_counts().reset_index().sort_values(by='count',ascending=False)
print(race)
# affectation manuel de la concordance pour plus de clareté. attention si les données changent il faut changer ici
race['str_race'] = ['Caucasian','Asian/Pacific','Latino','Other','African American']
print(race)
distrib_wave = px.bar(race,x='str_race',y='count', title='Distribution sur etnies')
distrib_wave.show()

   race  count
0   2.0   3786
1   4.0   1649
2   3.0    569
3   6.0    446
4   1.0    308
   race  count          str_race
0   2.0   3786         Caucasian
1   4.0   1649     Asian/Pacific
2   3.0    569            Latino
3   6.0    446             Other
4   1.0    308  African American


### Analyse des niveaux de revenus

In [81]:
incomes = df['income'].str.removesuffix('.00').str.replace(',','')
incomes = incomes.dropna()
print(f"La moyenne des revenus estimés est : {round(incomes.astype(float).mean(),2)} $")
print(f"La médiane des revenus estimés est : {round(incomes.astype(float).median(),2)} $")
incomes

La moyenne des revenus estimés est : 44887.61 $
La médiane des revenus estimés est : 43185.0 $


0       69487
1       69487
2       69487
3       69487
4       69487
        ...  
8351    55138
8352    55138
8353    55138
8354    55138
8355    55138
Name: income, Length: 4279, dtype: object

## Qui dit oui pour un second RDV le plus souvent ?

In [82]:
male = filterd_df[filterd_df['gender']==1]
female = filterd_df[filterd_df['gender']==0]

print(f"Les hommes ont dit {round((male['dec'] == 1).sum()/male['dec'].count()*100,2)} % oui à un second RDV")
print(f"Les femmes ont dit {round((female['dec'] == 1).sum()/female['dec'].count()*100,2)} % oui à un second RDV")
print(f"% de match : {round((df['match'] == 1).sum()/df['match'].count()*100,2)} %")

Les hommes ont dit 45.53 % oui à un second RDV
Les femmes ont dit 37.88 % oui à un second RDV
% de match : 16.47 %


## Quel est l'attribut le moins désirable

In [ ]:
female_attributes = female[['attr1_1','sinc1_1','intel1_1','fun1_1','amb1_1','shar1_1']]
print(female_attributes.mean().sort_values(ascending=True))
print("Pour les femmes, l'ambition chez un homme est en moyenne l'attribut le moins désirable")

print()
male_attributes = male[['attr1_1','sinc1_1','intel1_1','fun1_1','amb1_1','shar1_1']]
print(male_attributes.mean().sort_values(ascending=True))
print("Pour les hommes, l'ambition chez une femme est en moyenne l'attribut le moins désirable")


amb1_1      12.005635
shar1_1     12.264193
fun1_1      17.043556
sinc1_1     18.377089
attr1_1     18.788883
intel1_1    21.568459
dtype: float64
Pour les femmes, les centres d'interets communs chez un homme est en moyenne l'attribut le moins désirable

amb1_1       7.498773
shar1_1     10.258607
sinc1_1     16.233127
fun1_1      17.657753
intel1_1    19.564615
attr1_1     29.108895
dtype: float64
Pour les hommes, l'ambition chez une femme est en moyenne l'attribut le moins désirable


Pour les femmes, les centres d'interets communs chez un homme est en moyenne l'attribut le moins désirable.

Pour les hommes, l'ambition chez une femme est en moyenne l'attribut le moins désirable.

L'Attirance physique est en 1ère position chez les hommes alors qu'en 3ème chez les femmes.

Qu'en est il de la réalité ? 

In [ ]:
note_attribue_female_to_male = female[['attr','sinc','intel','fun','amb','shar','dec']]
note_attribue_male_to_female = male[['attr','sinc','intel','fun','amb','shar','dec']]

print("Pour les femmes : ")
reel_impact_femmes = note_attribue_female_to_male[note_attribue_female_to_male['dec'] == 1].mean().sort_values(ascending=False)
reel_impact_femmes = reel_impact_femmes.reset_index(name=('moyenne'))
print(reel_impact_femmes)
fig_women = px.bar(reel_impact_femmes,x='index',y="moyenne")
fig_women.show()
print()
print("Pour les hommes :")
reel_impact_hommes = note_attribue_male_to_female[note_attribue_male_to_female['dec'] == 1].mean().sort_values(ascending=False)
reel_impact_hommes = reel_impact_hommes.reset_index(name=('moyenne'))
print(reel_impact_hommes)
fig_men = px.bar(reel_impact_hommes,x='index',y="moyenne")
fig_men.show()


Pour les femmes : 
   index   moyenne
0  intel  7.883346
1   sinc  7.565574
2    fun  7.328370
3    amb  7.270938
4   attr  7.024903
5   shar  6.517016
6    dec  1.000000



Pour les hommes :
   index   moyenne
0  intel  7.883346
1   sinc  7.565574
2    fun  7.328370
3    amb  7.270938
4   attr  7.024903
5   shar  6.517016
6    dec  1.000000


L'attirance arrive en 5eme position quand les femmes ont dit oui à un seccond rdv

L'attirance arrive en 3eme position quand les hommes ont dit oui à un seccond rdv

### Y a t'il une corrélation entre le match et si les deux participants sont de la même etnie ?

In [85]:
batch = df.copy()
batch.columns


Index(['iid', 'id', 'gender', 'idg', 'condtn', 'wave', 'round', 'position',
       'positin1', 'order',
       ...
       'attr3_3', 'sinc3_3', 'intel3_3', 'fun3_3', 'amb3_3', 'attr5_3',
       'sinc5_3', 'intel5_3', 'fun5_3', 'amb5_3'],
      dtype='object', length=195)

In [86]:
df['goal'].head()

features = ['match','samerace','age','age_o','attr_o','position','field_cd','exphappy']
corr_df = df[features].corr('pearson')
corr_df_fig = px.imshow(corr_df, text_auto=True,
    color_continuous_scale="RdBu_r",
    title="Matrice de corrélation (Pearson)",
    zmin=-1, zmax=1)
corr_df_fig.show()

In [87]:
mean_attract_features_before_dating = ['attr1_1','sinc1_1','intel1_1','fun1_1','amb1_1','shar1_1']
print('Moyenne femmes et hommes confondus avant dating : ')
print(df[mean_attract_features_before_dating].set_axis(["Attirant", "Sincère", "Intelligent", "Drole", "Ambitieux","Interets commun"],axis='columns').mean())
print('-------')

## avec filtre gender
df_homme_b_dating = df[(df['gender']==1) & (df['wave'] != 6) & (df['wave'] != 7) & (df['wave'] != 8) & (df['wave'] != 9)]
df_femme_b_dating = df[(df['gender']==0) & (df['wave'] != 6) & (df['wave'] != 7) & (df['wave'] != 8) & (df['wave'] != 9)]

df_homme_b_dating = df_homme_b_dating[mean_attract_features_before_dating]
df_femme_b_dating = df_femme_b_dating[mean_attract_features_before_dating].set_axis(["Attirante", "Sincère", "Intelligente", "Drole", "Ambitieuse","Interets commun"],axis='columns')


print("Score critères important pour les femmes avant speed dating :")
print(df_femme_b_dating.mean())
print('-------')
print("Score critères important pour les hommes avant speed dating :")
print(df_homme_b_dating.mean())

Moyenne femmes et hommes confondus avant dating : 
Attirant           22.514632
Sincère            17.396389
Intelligent        20.265613
Drole              17.457043
Ambitieux          10.682539
Interets commun    11.845111
dtype: float64
-------
Score critères important pour les femmes avant speed dating :
Attirante          18.788883
Sincère            18.377089
Intelligente       21.568459
Drole              17.043556
Ambitieuse         12.005635
Interets commun    12.264193
dtype: float64
-------
Score critères important pour les hommes avant speed dating :
attr1_1     29.108895
sinc1_1     16.233127
intel1_1    19.564615
fun1_1      17.657753
amb1_1       7.498773
shar1_1     10.258607
dtype: float64


In [88]:
mean_attract_features = ['attr7_2','sinc7_2','intel7_2','fun7_2','amb7_2','shar7_2']
print('Moyenne femmes et hommes confondus : ')
print(df[mean_attract_features].set_axis(["Attirant", "Sincère", "Intelligent", "Drole", "Ambitieux","Interets commun"],axis='columns').mean())
print('-------')

## avec filtre gender
df_homme = df[(df['gender']==1) & (df['wave'] != 6) & (df['wave'] != 7) & (df['wave'] != 8) & (df['wave'] != 9)]
df_femme = df[(df['gender']==0) & (df['wave'] != 6) & (df['wave'] != 7) & (df['wave'] != 8) & (df['wave'] != 9)]

df_homme = df_homme[mean_attract_features].set_axis(["Attirant", "Sincère", "Intelligent", "Drole", "Ambitieux","partage_interet"],axis='columns')
df_femme = df_femme[mean_attract_features].set_axis(["Attirante", "Sincère", "Intelligente", "Drole", "Ambitieuse","partage_interet"],axis='columns')


print("Score critères important pour les femmes après speed dating :")
print(df_femme.mean())
print('-------')
print("Score critères important pour les hommes après speed dating :")
print(df_homme.mean())


Moyenne femmes et hommes confondus : 
Attirant           32.819556
Sincère            13.529923
Intelligent        15.293851
Drole              18.868448
Ambitieux           7.286957
Interets commun    12.156028
dtype: float64
-------
Score critères important pour les femmes après speed dating :
Attirante          28.630964
Sincère            15.263889
Intelligente       15.580331
Drole              18.426485
Ambitieuse          8.132937
partage_interet    13.779942
dtype: float64
-------
Score critères important pour les hommes après speed dating :
Attirant           37.314525
Sincère            11.684266
Intelligent        14.986416
Drole              19.342738
Ambitieux           6.386484
partage_interet    10.394931
dtype: float64
